### Market Basket Analysis 
DESKTOP-4P75PCB


In [2]:

import pyodbc 
import pandas as pd
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-4P75PCB;'
                      'Database=AdventureWorks2012;' 
                      'Trusted_Connection=yes;')


sql_for_df1 = """SELECT C.[SalesOrderID],A.[ProductID],A.[Name] AS ProductName,
                B.[Name] AS ProductSubCat,C.[OrderQty],C.[UnitPrice],D.[OrderDate],D.[OnlineOrderFlag] AS SalesChannel,D.CustomerID,F.Name AS Country
                              
               FROM [Production].[Product] AS A \
               FULL OUTER JOIN [Production].[ProductSubcategory] AS B \
               ON A.[ProductSubcategoryID] = B.[ProductSubcategoryID] \
               JOIN [Sales].[SalesOrderDetail] AS C \
               ON C.ProductID = A.ProductID \
               JOIN [Sales].[SalesOrderHeader] AS D \
               ON D.SalesOrderID = C.SalesOrderID \
               JOIN [Sales].[SalesTerritory] AS E \
               ON E.TerritoryID = D.TerritoryID \
               JOIN [Person].[CountryRegion] AS F \
               ON F.CountryRegionCode = E.CountryRegionCode"""


my_join = pd.read_sql(sql_for_df1 , conn)


conn.close()
my_join.head()

,SalesOrderID,ProductID,ProductName,ProductSubCat,OrderQty,UnitPrice,OrderDate,SalesChannel,CustomerID,Country
0,43659,776,"Mountain-100 Black, 42",Mountain Bikes,1,2024.994,2011-05-31,False,29825,United States
1,43659,777,"Mountain-100 Black, 44",Mountain Bikes,3,2024.994,2011-05-31,False,29825,United States
2,43659,778,"Mountain-100 Black, 48",Mountain Bikes,1,2024.994,2011-05-31,False,29825,United States
3,43659,771,"Mountain-100 Silver, 38",Mountain Bikes,1,2039.994,2011-05-31,False,29825,United States
4,43659,772,"Mountain-100 Silver, 42",Mountain Bikes,1,2039.994,2011-05-31,False,29825,United States


In [12]:
my_join.shape

(121317, 10)

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns #has slightly data visualisation

In [14]:
 my_join.SalesChannel = my_join.SalesChannel.map(lambda x: 1 if x == "True" else 0)

In [15]:
my_join.head()

,SalesOrderID,ProductID,ProductName,ProductSubCat,OrderQty,UnitPrice,OrderDate,SalesChannel,CustomerID,Country
0,43659,776,"Mountain-100 Black, 42",Mountain Bikes,1,2024.994,2011-05-31,0,29825,United States
1,43659,777,"Mountain-100 Black, 44",Mountain Bikes,3,2024.994,2011-05-31,0,29825,United States
2,43659,778,"Mountain-100 Black, 48",Mountain Bikes,1,2024.994,2011-05-31,0,29825,United States
3,43659,771,"Mountain-100 Silver, 38",Mountain Bikes,1,2039.994,2011-05-31,0,29825,United States
4,43659,772,"Mountain-100 Silver, 42",Mountain Bikes,1,2039.994,2011-05-31,0,29825,United States


In [24]:
my_join.shape

(121317, 10)

In [16]:
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

### Data Preprocessing.

In [25]:
my_join.isnull().sum() # doesnt have null value

SalesOrderID     0
ProductID        0
ProductName      0
ProductSubCat    0
OrderQty         0
UnitPrice        0
OrderDate        0
SalesChannel     0
CustomerID       0
Country          0
dtype: int64

In [26]:

my_join['SalesOrderID'] = my_join['SalesOrderID'].astype('str')


In [27]:
my_join['Country'].value_counts()

United States     60153
Canada            19064
Australia         15058
United Kingdom    10426
France             9088
Germany            7528
Name: Country, dtype: int64

In [ ]:
##Unites states has highest number of counts.

In [48]:
basket = my_join[['SalesOrderID', 'ProductID','ProductName','ProductSubCat','SalesChannel','Country','OrderQty']]
basket = basket.copy()
basket['OrderQty'] = basket['OrderQty'].astype('int')

In [49]:
basket = (basket[basket['Country'] =="Germany"].groupby(['SalesOrderID','ProductSubCat'])['OrderQty'].sum().unstack().reset_index().fillna(0).set_index('SalesOrderID'))

In [50]:
basket.head()

ProductSubCat,Bib-Shorts,Bike Racks,Bike Stands,Bottles and Cages,Bottom Brackets,Brakes,Caps,Chains,Cleaners,Cranksets,...,Road Frames,Saddles,Shorts,Socks,Tights,Tires and Tubes,Touring Bikes,Touring Frames,Vests,Wheels
SalesOrderID,,,,,,,,,,,,,,,,,,,,,
43712,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43720,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43725,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43737,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43752,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
basket.shape

(2623, 32)

In [52]:
basket.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2623 entries, 43712 to 75118
Data columns (total 32 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Bib-Shorts         2623 non-null   float64
 1   Bike Racks         2623 non-null   float64
 2   Bike Stands        2623 non-null   float64
 3   Bottles and Cages  2623 non-null   float64
 4   Bottom Brackets    2623 non-null   float64
 5   Brakes             2623 non-null   float64
 6   Caps               2623 non-null   float64
 7   Chains             2623 non-null   float64
 8   Cleaners           2623 non-null   float64
 9   Cranksets          2623 non-null   float64
 10  Derailleurs        2623 non-null   float64
 11  Fenders            2623 non-null   float64
 12  Gloves             2623 non-null   float64
 13  Handlebars         2623 non-null   float64
 14  Helmets            2623 non-null   float64
 15  Hydration Packs    2623 non-null   float64
 16  Jerseys            2623 

In [53]:
def one_hot_encode(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket = basket.applymap(one_hot_encode)
basket.head()


ProductSubCat,Bib-Shorts,Bike Racks,Bike Stands,Bottles and Cages,Bottom Brackets,Brakes,Caps,Chains,Cleaners,Cranksets,...,Road Frames,Saddles,Shorts,Socks,Tights,Tires and Tubes,Touring Bikes,Touring Frames,Vests,Wheels
SalesOrderID,,,,,,,,,,,,,,,,,,,,,
43712,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43720,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43725,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43737,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43752,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [90]:
frequent_itemsets = apriori(basket, min_support=0.05,low_memory=False,use_colnames=True)


In [91]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.5)
rules.head(5).sort_values(by='confidence')

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,(Helmets),(Caps),0.273732,0.120091,0.064811,0.236769,1.971570,0.031938,1.152873
0,(Helmets),(Bottles and Cages),0.273732,0.184903,0.070149,0.256267,1.385958,0.019535,1.095955
3,(Mountain Bikes),(Bottles and Cages),0.200152,0.184903,0.057949,0.289524,1.565816,0.020940,1.147255
2,(Bottles and Cages),(Mountain Bikes),0.184903,0.200152,0.057949,0.313402,1.565816,0.020940,1.164943
1,(Bottles and Cages),(Helmets),0.184903,0.273732,0.070149,0.379381,1.385958,0.019535,1.170232


In [92]:
rules[ (rules['lift'] >= 1) &
       (rules['confidence'] >= 0.1) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Helmets),(Bottles and Cages),0.273732,0.184903,0.070149,0.256267,1.385958,0.019535,1.095955
1,(Bottles and Cages),(Helmets),0.184903,0.273732,0.070149,0.379381,1.385958,0.019535,1.170232
2,(Bottles and Cages),(Mountain Bikes),0.184903,0.200152,0.057949,0.313402,1.565816,0.020940,1.164943
3,(Mountain Bikes),(Bottles and Cages),0.200152,0.184903,0.057949,0.289524,1.565816,0.020940,1.147255
4,(Helmets),(Caps),0.273732,0.120091,0.064811,0.236769,1.971570,0.031938,1.152873
5,(Caps),(Helmets),0.120091,0.273732,0.064811,0.539683,1.971570,0.031938,1.577754
6,(Helmets),(Mountain Bikes),0.273732,0.200152,0.056043,0.204735,1.022897,0.001254,1.005763
7,(Mountain Bikes),(Helmets),0.200152,0.273732,0.056043,0.280000,1.022897,0.001254,1.008705
10,(Tires and Tubes),(Helmets),0.342356,0.273732,0.108273,0.316258,1.155356,0.014559,1.062196
11,(Helmets),(Tires and Tubes),0.273732,0.342356,0.108273,0.395543,1.155356,0.014559,1.087992
